# Aprendizaje No Supervisado - Clustering Jerárquico y Reglas de Asociación
### Librerías y configuraciones

In [ ]:
# Importamos las librerías necesarias
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
import seaborn as sns
import copy
import matplotlib
import plotly.express as px
from PIL import Image
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

# Clustering Jerárquico
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc

# Reglas de Asociación
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.datasets import make_blobs

from joblib import dump, load
from sklearn.preprocessing import StandardScaler

rcParams['figure.figsize'] = (30,15)

## 2. Caso: Análisis de Cesta de Mercado

Para este caso, vamos a analizar un dataset con información del detalle de las compras realizadas en una cadena de Retail europea.

El objetivo del caso es identificar los set de ítems que son frecuentes en una compra de supermercado, identificando patrones de ocurrencia simultánea entre distintos productos. Luego, seleccionaremos qué **reglas** consideramos de interés para el negocio en función de las métricas de *soporte, confianza y lift*.

### 2.a) Lectura del set de datos

In [ ]:
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

### 2.b) Preparación de datos
Para poder realizar un análisis de reglas de asociación, necesitamos el set de datos en un formato particular, dependiendo de la librería utilizada.

Como en este caso estamos utilizando la librería *mlxtend*, necesitamos tener un registro por pedido (el numero de factura *InvoiceNo*) y en las columnas cada producto posible a incluir con un valor *True* o *False*, es decir, si lo incluye o no.

Para esto, primero realizamos una limpieza inicial quitando caracteres especiales y luego generamos una tabla Pivot.

In [ ]:
print(df.shape)

# Comenzamos cortando los espacios vacíos y pasando todo a mayúsculas
df['Description'] = df['Description'].str.strip().str.upper()

# Quitamos los pedidos sin ID porque se deben a errores
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

# Filtramos los pedidos donde la cantidad pedida sea <=0 ya que no son pedidos reales
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[df['Quantity'] > 0]

print(df.shape)

Ahora, agrupamos por pedido/producto para aproximar la salida a lo que necesitamos cuando utilicemos el algoritmo "apriori".

In [ ]:
# Quitamos los ítems DOTCOM POSTAGE y POSTAGE porque no son productos.
# Llenamos con 0 los pedidos donde un producto no fué comprado
df_group = (df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo').drop(["POSTAGE","DOTCOM POSTAGE"], axis=1))

# Seteamos True/False dependiendo de cada valor
df_group = df_group.applymap(lambda x: True if x >0 else False)

df_group

Vemos que tenemos un total de 20.136 pedidos con 4.058 productos distintos.

En caso de tener una fuente de datos con un formato menos amigable, se pueden hacer uso de las librerías de transformación de datos propias del modulo *mlxtend.preprocessing*, como es el caso del transformador *TransactionEncoder*.

#### Referencia: [Documentación de TransactionEncoder](http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/)
  



### 2.c) Construcción del modelo
Para construir un modelo de reglas de asociación necesitamos definir tres parámetros que van a influir directamente en el resultado obtenido y, sobre todo, en el tiempo de procesamiento. Estos parámetros son el *Soporte*, la *Confianza* y el *Lift*.

#### **Definición de itemset frecuente**
Antes de analizar las reglas de asociación entre todos los ítems, lo conveniente es definir un umbral mínimo de soporte, que permita considerar un *itemset* como **frecuente**. En nuestro caso, dado que tenemos 4058 productos consideramos un **umbral de soporte de 2.5%**.
Una vez identificados todos los itemsets frecuentes, podemos avanzar a la detección de reglas de asociación sólo de estos items, optimizando de forma exponencial el tiempo de procesamiento.

In [ ]:
x = 0.025

In [ ]:
frequent_itemsets = apriori(df_group, min_support=x, use_colnames=True)
frequent_itemsets.sort_values(by="support", ascending=False)

In [ ]:
frequent_itemsets.sample(1)

Vemos que el algoritmo obtuvo **220 itemsets más frecuentes**. Ahora debemos identificar qué cobertura queremos tener, y en función de eso definir el `umbral mínimo de soporte`.

#### **Interpretación de la métrica Soporte**
Para entender cómo se calcula la métrica de soporte, vamos a calcularlo de forma manual con un caso de ejemplo y ver como coincide con la métrica calculada.

Vamos a tomar un itemset de un solo elemento ("WHITE HANGING HEART T-LIGHT HOLDER") y uno compuesto de dos elementos ("CHARLOTTE BAG PINK POLKADOT", "RED RETROSPOT CHARLOTTE BAG").


In [ ]:
# Tomamos el soporte del itemset ('RED RETROSPOT CHARLOTTE BAG')
soporte_unq = frequent_itemsets[frequent_itemsets.itemsets == frozenset({'RED RETROSPOT CHARLOTTE BAG'})].support
print("Soporte del itemset ('RED RETROSPOT CHARLOTTE BAG'): {}".format(float(soporte_unq)))

# Tomamos el soporte del itemset ('CHARLOTTE BAG PINK POLKADOT', 'RED RETROSPOT CHARLOTTE BAG')
soporte_par = frequent_itemsets[frequent_itemsets.itemsets == frozenset({'CHARLOTTE BAG PINK POLKADOT', 'RED RETROSPOT CHARLOTTE BAG'})].support
print("Soporte del itemset ('CHARLOTTE BAG PINK POLKADOT', 'RED RETROSPOT CHARLOTTE BAG'): {}".format(float(soporte_par)))


Ahora veamos cuántos pedidos incluyen a cada itemset, y qué porcentaje del total de pedidos representan.

In [ ]:
# Cantidad total de pedidos
cn_total = df_group.shape[0]
print("Cantidad Total de Pedidos: {}".format(int(cn_total)))
print("-"*100)

# Comenzamos por el itemset simple  ('RED RETROSPOT CHARLOTTE BAG')
cn_unq   = df_group[df_group['RED RETROSPOT CHARLOTTE BAG']==True].shape[0]

print("Cantidad Pedidos que incluyen a ('RED RETROSPOT CHARLOTTE BAG'): {}".format(int(cn_unq)))
print("Soporte de ('RED RETROSPOT CHARLOTTE BAG'): {}".format(float(cn_unq/cn_total)))
print("-"*100)

# Ahora lo mismo para el itemset complejo ('CHARLOTTE BAG PINK POLKADOT', 'RED RETROSPOT CHARLOTTE BAG')
# Para este caso necesitamos que ambos items se encuentren en el mismo pedido simultaneamente
cn_unq   = df_group[ ( df_group['RED RETROSPOT CHARLOTTE BAG']==True ) & ( df_group['CHARLOTTE BAG PINK POLKADOT']==True )].shape[0]

print("Cantidad Pedidos que incluyen a ('CHARLOTTE BAG PINK POLKADOT', 'RED RETROSPOT CHARLOTTE BAG'): {}".format(int(cn_unq)))
print("Soporte de ('CHARLOTTE BAG PINK POLKADOT', 'RED RETROSPOT CHARLOTTE BAG'): {}".format(float(cn_unq/cn_total)))

Podemos ver que coincide con lo calculado por el algoritmo de apriori. Esto nos sirve como referencia para entender qué umbral de soporte queremos utilizar en función de la cobertura esperada de las reglas obtenidas.

Por ejemplo, teniendo **20.136 pedidos** y habiendo indicador un **soporte mínimo de 2,5% (0.025)**, nos aseguramos que cualquier itemset que analizaremos, será aplicable **por lo menos en 503 pedidos**.

La forma general de calcular la cobertura es:
```
COBERTURA = CANTIDAD TOTAL DE IDS * SOPORTE
```

#### **Detección de Reglas de Asociación**

Definidos los itemsets frecuentes, realizamos el análisis de reglas de asociación y comparamos distintas métricas, a fin de identificar cuáles son de utilidad para el negocio.

En este caso vamos a usar como métrica LIFT y a considerar todas las reglas con LIFT mayor a 1, es decir, que tengan alguna variación con respecto a la distribución estándar de ocurrencia de productos.

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Ordenamos por confianza de mayor a menor
rules.sort_values(by="confidence", ascending=False).head(5)

In [ ]:
rules.shape

### 2.d) Interpretación del Soporte y la Confianza


*   La métrica de **soporte** de una regla, explica qué *cobertura* tiene la regla detectada dentro de *total de observaciones*.

*   La métrica de **confianza** de una regla, explica qué cobertura tiene la regla detectada sobre el *antecedente* en forma porcentual.

Veamos un  ejemplo, si definimos una regla R1:
```
R1 : A -> B
```
siendo A el antecedente y B el consecuente.
<br>
- **Soporte R1**: Proporción de pedidos donde se encuentran A y B sobre el total de pedidos.

  `Soporte R1:{A->B} =  (#Ocurrencias A^B)  / (#Observaciones) `
<br>
- **Confianza R1**: Proporción de pedidos donde se encuentran A y B sobre el total de pedidos donde se encuentra A:

  `confianza R1:{A->B} = (#Ocurrencias A^B) / (#Ocurrencias A)  `
<br>

Verificamos la métrica calculada por el algoritmo para la regla de mayor `Confianza`:

```
Regla: ('PINK REGENCY TEACUP AND SAUCER','ROSES REGENCY TEACUP AND SAUCER') -> ('GREEN REGENCY TEACUP AND SAUCER')
```



In [ ]:
# Cantidad total de pedidos
print("Cantidad Total de Pedidos: {}".format(int(cn_total)))
print(" "*100)
print("-"*100)
print(" "*100)

# Analizamos el soporte del antecedente  ('PINK REGENCY TEACUP AND SAUCER','ROSES REGENCY TEACUP AND SAUCER')
cn_antecedente   = df_group[ ( df_group['PINK REGENCY TEACUP AND SAUCER']==True ) &
                             ( df_group['ROSES REGENCY TEACUP AND SAUCER']==True )].shape[0]

print("Cantidad Pedidos que incluyen a ('PINK REGENCY TEACUP AND SAUCER','ROSES REGENCY TEACUP AND SAUCER') : {}".format(int(cn_antecedente)))
print("Soporte del antecedente ('PINK REGENCY TEACUP AND SAUCER','ROSES REGENCY TEACUP AND SAUCER'): {}".format(float(cn_antecedente/cn_total)))
print(" "*100)
print("-"*100)

# Analizamos el soporte de la regla, es decir, la ocurrencia conjunta del consecuente y el antecedente
cn_regla   = df_group[ ( df_group['PINK REGENCY TEACUP AND SAUCER']==True ) &
                       ( df_group['ROSES REGENCY TEACUP AND SAUCER']==True ) &
                       ( df_group['GREEN REGENCY TEACUP AND SAUCER']==True )].shape[0]
print(" "*100)
print("R: ('PINK REGENCY TEACUP AND SAUCER','ROSES REGENCY TEACUP AND SAUCER') -> ('GREEN REGENCY TEACUP AND SAUCER')")
print(" "*100)
print("Cantidad Pedidos que cubre la regla : {}".format(int(cn_regla)))
print("Soporte de la regla: {}".format(float(cn_regla/cn_total)))

# Ahora calculamos la confianza, como la cantidad de pedidos donde ocurre la regla, sobre los pedidos del antecedente
print("Confianza de la regla: {}".format(float(cn_regla/cn_antecedente)))

Podemos concluir que **cerca del 3%** de los pedidos incluyen los productos **'PINK REGENCY TEACUP AND SAUCER', 'ROSES REGENCY TEACUP AND SAUCER'**, y dentro de esos pedidos **el 90%** también incluyen el producto **'GREEN REGENCY TEACUP AND SAUCER'**.

Básicamente, estamos identificando las personas que compran juegos de té (Té Verde, Té Rosa, Té de rosas), si compró dos juegos de té, es muy problable que compre una tercera. Esto puede estar también relacionado con la típica promoción 3x2 o algo similar.

### 2.e) Interpretación de la métrica LIFT

El **LIFT** es una métrica que nos da una idea de qué tan relevante es el patrón detectado por una regla. La relevancia es medida comparando la confianza de la regla con el soporte del consecuente.

  `LIFT R1:{A->B} = Confianza R1 / soporte{B}   `

Entonces, si no existiera correlación de ocurrencia entre los itemsets esta relacion LIFT debería ser 1.

En resumen, la interpretación del LIFT indica que:

 * Si el LIFT = 1, no existe relación de ocurrencia entre A y B.

 * Si el LIFT > 1, existe una relación positiva de ocurrencia entre A y B. Es más probable encontrar los productos juntos que por separado.

 * Si el LIFT < 1, existe una relación negativa de ocurrencia entre A y B, es más probable encontrarlos por separado que en conjunto.

### 2.f) Explotación de Reglas Obtenidas

Una vez generado el modelo, podemos seleccionar las reglas que más nos interesen ordenadas por Lift/Confianza/Soporte y también consultando aquellas que incluyan productos que resulten de interés comercial.

Por ejemplo, queremos mejorar las ventas de cerveza Patagonia, podemos analizar qué productos suelen venderse en conjunto y a aquellos clientes que compraron alguno de esos productos y aún no compraron cerveza Patagonia, les hacemos una oferta pro-activa.

Este tipo de acciones hoy en día se realizan con sistemas de recomendación más versátiles, pero en el fondo siguen embebiendo reglas que analizan la ocurrencia conjunta, ya sea de pedidos o de consumidores.

In [ ]:
# Reglas con soporte mayor a 3.5%, confianza > 70% y lift > 2
rules[(rules.support > 0.035) & (rules.lift > 2) & (rules.confidence > 0.70)]